## Snowflake notebook

**Goal:** Creates tables, test data, and inserts that test data into one of the tables

In [ ]:
CREATE OR REPLACE TABLE all_workouts (
    workout_id INT AUTOINCREMENT PRIMARY KEY, -- Unique identifier for each workout
    workout_name VARCHAR(100),               -- Name of the workout
    workout_type VARCHAR(50),                -- Type of the workout (e.g., cardio, HIIT)
    trainer_name VARCHAR(50)                 -- Trainer associated with the workout
);

-- Insert dummy data
INSERT INTO all_workouts (workout_name, workout_type, trainer_name) VALUES
('Cardio Blast', 'Cardio', 'Tony Horton'),
('HIIT Burnout', 'HIIT', 'Tony Horton'),
('Strength Circuit', 'Strength', 'Tony Horton'),
('Endurance Max', 'Endurance', 'Tony Horton'),
('Hypertrophy Builder', 'Hypertrophy', 'Jackson Bloor'),
('Stretch and Recover', 'Stretching', 'Jackson Bloor'),
('HIIT Power', 'HIIT', 'Jackson Bloor'),
('Functional Fitness', 'Strength', 'Jackson Bloor'),
('Muscle Growth Focus', 'Hypertrophy', 'Sagi Kalev'),
('Ultimate Flexibility', 'Stretching', 'Sagi Kalev');


--



In [ ]:
CREATE OR REPLACE TABLE WORKOUT_DETAILS (
    exercise_id INT,
    workout_id INT,
    exercise_name STRING,
    exercise_description STRING,
    critical_exercise_selection BOOLEAN,
    exercise_classification STRING,
    strength BOOLEAN,
    endurance BOOLEAN,
    flexibility BOOLEAN,
    balance BOOLEAN,
    cardio BOOLEAN,
    power BOOLEAN,
    primary_movement BOOLEAN,
    current_row BOOLEAN,
    from_date TIMESTAMP,
    to_date TIMESTAMP,
    profiled_by STRING
)

## now we create some test data and put it in a pandas df

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Step 2: Generate Test Data
def generate_exercise_data(workout_id, num_exercises, critical_range, primary_movements, instructor_name):
    exercises = []
    current_date = datetime.now()
    workout_start_date = current_date - timedelta(days=random.randint(10, 60))
    exercise_names = [
        "Squats", "Push-ups", "Deadlifts", "Bench Press", "Plank Holds", "Lunges",
        "Dumbbell Rows", "Overhead Press", "Burpees", "Mountain Climbers", "Jumping Jacks",
        "Bicep Curls", "Tricep Dips", "Pull-ups", "Leg Press", "Cable Flys"
    ]
    classifications = ["Legs", "Chest", "Arms", "Back", "Cardio", "Bodyweight"]
    dimensions = ["strength", "endurance", "flexibility", "balance", "cardio", "power"]
    
    for i in range(1, num_exercises + 1):
        exercise_id = (workout_id - 1) * 10 + i  # Unique ID logic
        exercise_name = random.choice(exercise_names)
        exercise_desc = f"{random.randint(3, 5)} sets of {random.randint(8, 15)} reps."
        is_critical = random.choice([True] * critical_range + [False] * (10 - critical_range))
        classification = random.choice(classifications)
        is_primary = exercise_name in primary_movements
        dim_flags = {dim: False for dim in dimensions}
        
        if is_primary:
            selected_dims = random.sample(dimensions, 2)
            for dim in selected_dims:
                dim_flags[dim] = True
        elif is_critical:
            selected_dim = random.choice(dimensions)
            dim_flags[selected_dim] = True
        
        # Historical and Current rows
        exercises.append({
            "exercise_id": exercise_id,
            "workout_id": workout_id,
            "exercise_name": exercise_name,
            "exercise_description": exercise_desc,
            "critical_exercise_selection": is_critical,
            "exercise_classification": classification,
            **dim_flags,
            "primary_movement": is_primary,
            "current_row": False,
            "from_date": workout_start_date.strftime('%Y-%m-%d %H:%M:%S'),
            "to_date": (workout_start_date + timedelta(days=random.randint(20, 40))).strftime('%Y-%m-%d %H:%M:%S'),
            "profiled_by": instructor_name
        })
        exercises.append({
            "exercise_id": exercise_id,
            "workout_id": workout_id,
            "exercise_name": exercise_name,
            "exercise_description": exercise_desc,
            "critical_exercise_selection": is_critical,
            "exercise_classification": classification,
            **dim_flags,
            "primary_movement": is_primary,
            "current_row": True,
            "from_date": (workout_start_date + timedelta(days=random.randint(20, 40))).strftime('%Y-%m-%d %H:%M:%S'),
            "to_date": "9999-12-31 23:59:59",
            "profiled_by": instructor_name
        })
    
    return exercises

# Generate data for all workouts
workout_ids = range(1, 11)
primary_movements = ["Squats", "Deadlifts", "Bench Press"]
instructor_name = "Automated_Ghost_Of_Jack_LaLanne"
data = []

for workout_id in workout_ids:
    data.extend(generate_exercise_data(
        workout_id=workout_id,
        num_exercises=random.randint(8, 10),
        critical_range=random.randint(4, 6),
        primary_movements=primary_movements,
        instructor_name=instructor_name
    ))

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to verify
print(df.head())


## now we create a snowpark df out of this pandas dataframe

In [ ]:


#convert pandas df to snowpark df (so I can write this into the snowflake table)
from snowflake.snowpark import Session, Row
from snowflake.snowpark import types as T
from snowflake.snowpark.context import get_active_session
session = get_active_session() 

schema = T.StructType([
    T.StructField("exercise_id", T.IntegerType()),
    T.StructField("workout_id", T.IntegerType()),
    T.StructField("exercise_name", T.StringType()),
    T.StructField("exercise_description", T.StringType()),
    T.StructField("critical_exercise_selection", T.BooleanType()),
    T.StructField("exercise_classification", T.StringType()),
    T.StructField("strength", T.BooleanType()),
    T.StructField("endurance", T.BooleanType()),
    T.StructField("flexibility", T.BooleanType()),
    T.StructField("balance", T.BooleanType()),
    T.StructField("cardio", T.BooleanType()),
    T.StructField("power", T.BooleanType()),
    T.StructField("primary_movement", T.BooleanType()),
    T.StructField("current_row", T.BooleanType()),
    T.StructField("from_date", T.StringType()),
    T.StructField("to_date", T.StringType()),
    T.StructField("profiled_by", T.StringType())
])




sp_df = session.create_dataframe(df, schema=schema)




## now we insert this snowpark df into the table.

In [ ]:
sp_df.write.mode("overwrite").save_as_table("WORKOUT_DETAILS")

In [ ]:
select * from WORKOUT_DETAILS;